# Text generation test

In [4]:
from vertexai.preview.language_models import TextGenerationModel
import tomllib

In [121]:
with open("config.toml", "rb") as f:
    config = tomllib.load(f)

In [6]:
text_gen_client = TextGenerationModel.from_pretrained(model_name="text-bison")
prompt_email_support = config["global"]["prompt_email_support"]
exception_email_response = config["global"]["exception_email_response"]
exception_nosummary_email_response = config["global"]["exception_nosummary_email_response"]

In [7]:
email = """hi support,
I writing to let you know that I am disappointed with the product documentation.
I need to know if gaming chairs are ergonomic or not.
Could you please help me with this question?
I also want to know if that's valid for nylon chairs too.
Thank you
Best"""

In [8]:
questions = text_gen_client.predict(
    prompt=prompt_email_support.format(email),
    max_output_tokens=1024,
    temperature=0.2,
    top_k=40,
    top_p=0.8
).text

In [9]:
questions

' {\n    "questions": [\n        "Are gaming chairs ergonomic?",\n        "Are nylon chairs ergonomic?"\n    ]\n}'

In [10]:
import json
questions_dict = json.loads(questions)

In [11]:
questions_dict

{'questions': ['Are gaming chairs ergonomic?', 'Are nylon chairs ergonomic?']}

In [13]:
"questions" not in questions_dict

False

In [16]:
not isinstance(questions_dict["questions"], list)

False

In [15]:
not questions_dict["questions"]

False

# Search HTML

In [1]:
from google.cloud import discoveryengine_v1beta as discoveryengine
import tomllib

with open("config.toml", "rb") as f:
    config = tomllib.load(f)

In [2]:
text_gen_client = TextGenerationModel.from_pretrained(model_name="text-bison")
prompt_email_support = config["global"]["prompt_email_support"]
exception_email_response = config["global"]["exception_email_response"]
exception_nosummary_email_response = config["global"]["exception_nosummary_email_response"]

email_datastore_id = config["global"]["email_datastore_id"]
datastore_location = config["global"]["datastore_location"]
search_client = discoveryengine.SearchServiceClient()


NameError: name 'TextGenerationModel' is not defined

In [5]:
converse_client = discoveryengine.ConversationalSearchServiceClient()

serving_config = converse_client.serving_config_path(
    project="rl-llm-dev",
    location="global",
    data_store="csm-products_1696914795319",
    serving_config="default_config")

In [6]:
converse_request = discoveryengine.CreateConversationRequest(
    parent = f"projects/rl-llm-dev/locations/global/collections/default_collection/dataStores/csm-products_1696914795319",
    conversation=discoveryengine.Conversation(
        user_pseudo_id="egonsoares"
    )
)
conversation = converse_client.create_conversation(request=converse_request)

In [7]:
request = discoveryengine.ConverseConversationRequest(
    name=conversation.name,
    query=discoveryengine.TextInput(input="gaming chairs"),
    serving_config=serving_config)

In [9]:
converse_client.converse_conversation(request)

reply {
  reply: "There are many different gaming chairs available on Google Shopping. Some of the most popular models include the JOYFLY Gaming Chairs with Footrest, the Vigosit Gaming Chair, and the GTRACING Gaming Chair with Footrest Speakers. These chairs all offer a variety of features, such as adjustable backrests, headrests, and lumbar support, to help you find the most comfortable position for gaming."
  summary {
    summary_text: "There are many different gaming chairs available on Google Shopping. Some of the most popular models include the JOYFLY Gaming Chairs with Footrest, the Vigosit Gaming Chair, and the GTRACING Gaming Chair with Footrest Speakers. These chairs all offer a variety of features, such as adjustable backrests, headrests, and lumbar support, to help you find the most comfortable position for gaming."
    safety_attributes {
      categories: "Health"
      categories: "Insult"
      categories: "Toxic"
      scores: 0.1
      scores: 0.1
      scores: 0.1
 

In [19]:
import utils_search

In [27]:
search_results = utils_search.search_for_salesforce(
        search_client=search_client,
        search_query=questions_dict["questions"][0],
        project_id="rl-llm-dev",
        datastore_location=datastore_location,
        datastore_id=email_datastore_id)

In [28]:
from proto import Message

In [37]:
"[1]" in Message.to_dict(search_results.summary)["summary_text"]

True

In [68]:
for i in search_results.results:
    print(Message.to_dict(i)["document"]["derived_struct_data"]["link"])

gs://csm-dataset/website-search/5.html
gs://csm-dataset/website-search/6.html
gs://csm-dataset/website-search/3.html
gs://csm-dataset/website-search/4.html
gs://csm-dataset/website-search/2.html
gs://csm-dataset/website-search/9.html
gs://csm-dataset/website-search/7.html
gs://csm-dataset/website-search/1.html
gs://csm-dataset/website-search/8.html
gs://csm-dataset/website-search/0.html


In [51]:
Message.to_dict(search_results.results[0])["document"]["derived_struct_data"]["link"]

'gs://csm-dataset/website-search/5.html'

In [94]:
results = []
for question in questions_dict["questions"]:
    result = {}
    search_results = utils_search.search_for_salesforce(
        search_client=search_client,
        search_query=question,
        project_id="rl-llm-dev",
        datastore_location=datastore_location,
        datastore_id=email_datastore_id)

    search_result_dict = Message.to_dict(search_results.summary)
    summary_text = search_result_dict.get("summary_text", {})

    if not summary_text:
        continue

    result["question"] = question
    result["summary_text"] = summary_text
    result["links"] = []

    for i, r in enumerate(search_results.results):
        link = Message.to_dict(r)[
            "document"]["derived_struct_data"]["link"]
        if i < 2:
            result["links"].append(f"[{i+1}] " + link)
        else:
            break

    results.append(result)

In [127]:
email_response = f"""Dear Customer,
Thank you for contacting the Cymbal support team.
We identified some questions in your inquiry. We hope the following answers can be of help to you:

"""

for i in results:
    joined_links = '\n'.join(i["links"])
    summary = i["summary_text"] or "Here are some links you might find useful."

    email_response += f"""**Question**: {i["question"]}
**Answer**: {summary}
{joined_links} \n
"""

In [128]:
print(email_response)

Dear Customer,
Thank you for contacting the Cymbal support team.
We identified some questions in your inquiry. We hope the following answers can be of help to you:

**Question**: Are gaming chairs ergonomic?
**Answer**: Yes, gaming chairs are ergonomically designed to support your back, protect your spine and cervical spine, and help you develop a scientific and healthy sitting posture while providing effective relief from your back pain [1].
[1] gs://csm-dataset/website-search/5.html
[2] gs://csm-dataset/website-search/6.html 

**Question**: Are nylon chairs ergonomic?
**Answer**: I found some results that mention nylon chairs. However, none of them mention whether nylon chairs are ergonomic or not. I'm not sure if nylon chairs are ergonomic or not.
[1] gs://csm-dataset/website-search/5.html
[2] gs://csm-dataset/website-search/6.html 




In [129]:
from googleapiclient.discovery import build
from email.message import EmailMessage
from google.oauth2.credentials import Credentials

# Workspace integration
CREDENTIALS = Credentials.from_authorized_user_file('token.json', config["global"]["workspace_scopes"])

gmail_services = build('gmail', 'v1', credentials=CREDENTIALS)

In [131]:
from email.mime.text import MIMEText

In [132]:
import base64
# message = EmailMessage()
message = MIMEText(email_response, 'html')

# message.set_content(email_response)

message['To'] = 'renatoleite@google.com'
message['From'] = 'renatoleite@1987984870407.altostrat.com'
message['Subject'] = "About your questions"

# encoded message
encoded_message = base64.urlsafe_b64encode(message.as_bytes()) \
    .decode()

create_message = {
    'raw': encoded_message
}
# pylint: disable=E1101
gmail_services.users().messages().send(
    userId="renatoleite@1987984870407.altostrat.com", 
    body=create_message).execute()

{'id': '18b1c425fb4c09c0',
 'threadId': '18b1c425fb4c09c0',
 'labelIds': ['SENT']}